# Working with StreamSets

In this notebook we will focus on retrieving data from groups of time series streams (a `StreamSet`) as opposed to concentrating on individual streams.  You can think of a `StreamSet` as a wrapper around a list of `Stream` objects as found in the preceding notebook.

The StreamSet class has a number of methods to mirror those found in regular Stream objects including methods for transforming and serializing the data to different formats. 

As with a `Stream`, retrieving data from the BTrDB server will fully materialize in memory so please keep this in mind.  In other words, do not attempt to retreive data that is greater than the amount of memory available to you.

If you would like to learn more about any of the topics covered here, please see the btrdb library [documentation](https://btrdb.readthedocs.io/en/develop/index.html).

**NOTE**: To get access to the Sunshine dataset to run this notebook, please register for an API key at [ni4ai.org](https://ni4ai.org/).

## Imports

In [1]:
import btrdb
import yaml
from tabulate import tabulate
from btrdb.utils.timez import ns_delta

## Connect To Server

To get started we'll connect to the server and define a helper method from a previous notebook.

In [2]:
# Make sure you add your API key to the config file to connect!
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
conn = btrdb.connect(config['connection']['api_url'], config['connection']['api_key'])
conn.info()

{'majorVersion': 5, 'build': '5.11.37', 'proxy': {'proxyEndpoints': []}}

In [3]:
def describe_streams(streams):
    table = [["Collection", "Name", "Units", "Version", "Earliest", "Latest"]]
    for stream in streams:
        tags = stream.tags()
        table.append([
            stream.collection, stream.name, tags["unit"], stream.version(), 
            stream.earliest()[0].time, stream.latest()[0].time, 
        ])
    return tabulate(table, headers="firstrow")

# Helper Methods

The best way to think about the `StreamSet` is as a wrapper around a list of `Stream` objects with appropriate methods added to help with examining your data. To create a `StreamSet` we first need the UUIDs for streams, which we will obtain by selecting a few voltage streams from the database.

In [4]:
streams = conn.streams_in_collection('sunshine/PMU2', tags={"unit": "amps"})
streams = list(streams)
print(describe_streams(streams))

Collection     Name    Units      Version             Earliest               Latest
-------------  ------  -------  ---------  -------------------  -------------------
sunshine/PMU2  C2MAG   amps         19018  1456790400008333000  1464738948999999000
sunshine/PMU2  C1MAG   amps         19018  1456790400008333000  1464738948999999000
sunshine/PMU2  C3MAG   amps         19018  1456790400008333000  1464738948999999000


Now let's create a `StreamSet` using the UUIDs from our known streams.  In the future, we will be able to query based on collection, tags, etc., but for now we need to provide the UUIDs.

In [5]:
UUIDs = [s.uuid for s in streams]
UUIDs

[UUID('7925a0be-536e-44f0-875b-e176a7412f14'),
 UUID('a6df7f78-5b6d-4036-a7ef-4de95d04a53e'),
 UUID('9cc07564-7fa2-4961-8cda-1bb4f469317d')]

In [6]:
streamset = conn.streams(*UUIDs)
streamset

<StreamSet(3 streams)>

Now that we have a StreamSet with the four streams, let's take a look at some of the helper methods.  In a Stream object, the `earliest` method would provide the first point in the Stream.  `StreamSet.earliest` will provide a tuple containing the first points from each individual stream.  The order is the same as the UUIDs that were provided when creating the instance.

In [7]:
streamset.earliest()

(RawPoint(1456790400008333000, 115.00888061523438),
 RawPoint(1456790400008333000, 115.58443450927734),
 RawPoint(1456790400008333000, 104.03299713134766))

Similarly, let's look at the latest points in the streams.

In [8]:
streamset.latest()

(RawPoint(1464738948999999000, 94.16214752197266),
 RawPoint(1464738948999999000, 99.10070037841797),
 RawPoint(1464738948999999000, 89.8238525390625))

# Viewing Data

Like the `Stream` object, the `StreamSet` has a `values` method which will return a list of lists.  Each internal list contains the `RawPoint` instances for a given stream.  Just as before we will return only a little bit of the data from the beginning of the streams.

We will start by finding the earliest time from all of the earliest points although in this case they all have the same beginning time.

In [9]:
earliest_point = sorted(streamset.earliest(), key=lambda p: p.time)[0]
earliest_point.time

1456790400008333000

Next we will ask for the values in the streams. Stream values are returned as a list of list of points such that the lists of points are ordered according to the UUIDs provided on initialization. Using this method data is fetched for each stream and returned and can be thought of as a helper method to query multiple streams simultaneously.

In [10]:
start = earliest_point.time
end = start + ns_delta(milliseconds=100)
streamset.filter(start, end).values()

[[RawPoint(1456790400008333000, 115.00888061523438),
  RawPoint(1456790400016666000, 114.9324722290039),
  RawPoint(1456790400024999000, 115.0665512084961),
  RawPoint(1456790400033333000, 115.17729949951172),
  RawPoint(1456790400041666000, 114.99266815185547),
  RawPoint(1456790400049999000, 114.45643615722656),
  RawPoint(1456790400058333000, 113.85437774658203),
  RawPoint(1456790400066666000, 113.72930145263672),
  RawPoint(1456790400074999000, 113.93565368652344),
  RawPoint(1456790400083333000, 113.95361328125),
  RawPoint(1456790400091666000, 113.69194030761719),
  RawPoint(1456790400099999000, 113.29793548583984)],
 [RawPoint(1456790400008333000, 115.58443450927734),
  RawPoint(1456790400016666000, 115.8336410522461),
  RawPoint(1456790400024999000, 116.0658950805664),
  RawPoint(1456790400033333000, 115.89474487304688),
  RawPoint(1456790400041666000, 115.30825805664062),
  RawPoint(1456790400049999000, 114.27113342285156),
  RawPoint(1456790400058333000, 113.3078842163086),


You may have noticed that we first called the `filter` method and then called the `values` method with no arguments.  The `StreamSet` class was designed to support a method chaining style of programming and so behaves slightly differently from the `Stream`.

Data is only ever materialized when calling the `values` or `rows` method as demonstrated below.  The `rows` method is similar to the `values` method but orients the data differently.  Here you will notice that the streams are aligned according to time.  The first tuple contains all of the data for the first time index.  If any streams do not have data at that time index, then `None` is used as a placeholder.

In [11]:
streamset.filter(start, end).rows()

[(RawPoint(1456790400008333000, 115.00888061523438),
  RawPoint(1456790400008333000, 115.58443450927734),
  RawPoint(1456790400008333000, 104.03299713134766)),
 (RawPoint(1456790400016666000, 114.9324722290039),
  RawPoint(1456790400016666000, 115.8336410522461),
  RawPoint(1456790400016666000, 103.94195556640625)),
 (RawPoint(1456790400024999000, 115.0665512084961),
  RawPoint(1456790400024999000, 116.0658950805664),
  RawPoint(1456790400024999000, 103.55602264404297)),
 (RawPoint(1456790400033333000, 115.17729949951172),
  RawPoint(1456790400033333000, 115.89474487304688),
  RawPoint(1456790400033333000, 103.05726623535156)),
 (RawPoint(1456790400041666000, 114.99266815185547),
  RawPoint(1456790400041666000, 115.30825805664062),
  RawPoint(1456790400041666000, 102.64067840576172)),
 (RawPoint(1456790400049999000, 114.45643615722656),
  RawPoint(1456790400049999000, 114.27113342285156),
  RawPoint(1456790400049999000, 102.25871276855469)),
 (RawPoint(1456790400058333000, 113.85437774

Let's use the tabulate library again to better format the data rows.

In [12]:
table = [["time"] + [s.name for s in streamset]]

for row in streamset.filter(start, end).rows():
    time = sorted([p.time for p in row])[-1]
    data = [time]
    for point in row:
        data.append(point.value)
    table.append(data)
        
print(tabulate(table, headers="firstrow"))

               time    C2MAG    C1MAG    C3MAG
-------------------  -------  -------  -------
1456790400008333000  115.009  115.584  104.033
1456790400016666000  114.932  115.834  103.942
1456790400024999000  115.067  116.066  103.556
1456790400033333000  115.177  115.895  103.057
1456790400041666000  114.993  115.308  102.641
1456790400049999000  114.456  114.271  102.259
1456790400058333000  113.854  113.308  102.027
1456790400066666000  113.729  113.053  102.049
1456790400074999000  113.936  113.111  102.063
1456790400083333000  113.954  113.043  101.942
1456790400091666000  113.692  112.812  101.871
1456790400099999000  113.298  112.506  101.874


# Transforming Data to Other Formats

A number of methods have been provided to convert the point data objects into objects you may already be familiar with such as numpy arrays and pandas dataframes.  Using these transformation methods materializes the data similar to the `values` method.  Examples of the available transformations follow.

## Numpy Arrays

Converting to Numpy arrays will produce a list of arrays.  This output will be similar in structure to calling the `values` method.

In [13]:
start = earliest_point.time
end = start + ns_delta(milliseconds=100)

streamset.filter(start, end).to_array()

array([[115.00888062, 114.93247223, 115.06655121, 115.1772995 ,
        114.99266815, 114.45643616, 113.85437775, 113.72930145,
        113.93565369, 113.95361328, 113.69194031, 113.29793549],
       [115.58443451, 115.83364105, 116.06589508, 115.89474487,
        115.30825806, 114.27113342, 113.30788422, 113.05310822,
        113.11147308, 113.04302216, 112.8117218 , 112.505867  ],
       [104.03299713, 103.94195557, 103.55602264, 103.05726624,
        102.64067841, 102.25871277, 102.02741241, 102.04944611,
        102.06348419, 101.94192505, 101.87135315, 101.87373352]])

## Pandas Series

Converting to a pandas series will produce a view of the data similar to calling the `values` method.  The resulting series will be indexed by time.

In [14]:
streamset.filter(start, end).to_series()

[2016-03-01 00:00:00.008333    115.008881
 2016-03-01 00:00:00.016666    114.932472
 2016-03-01 00:00:00.024999    115.066551
 2016-03-01 00:00:00.033333    115.177299
 2016-03-01 00:00:00.041666    114.992668
 2016-03-01 00:00:00.049999    114.456436
 2016-03-01 00:00:00.058333    113.854378
 2016-03-01 00:00:00.066666    113.729301
 2016-03-01 00:00:00.074999    113.935654
 2016-03-01 00:00:00.083333    113.953613
 2016-03-01 00:00:00.091666    113.691940
 2016-03-01 00:00:00.099999    113.297935
 Name: sunshine/PMU2/C2MAG, dtype: float64,
 2016-03-01 00:00:00.008333    115.584435
 2016-03-01 00:00:00.016666    115.833641
 2016-03-01 00:00:00.024999    116.065895
 2016-03-01 00:00:00.033333    115.894745
 2016-03-01 00:00:00.041666    115.308258
 2016-03-01 00:00:00.049999    114.271133
 2016-03-01 00:00:00.058333    113.307884
 2016-03-01 00:00:00.066666    113.053108
 2016-03-01 00:00:00.074999    113.111473
 2016-03-01 00:00:00.083333    113.043022
 2016-03-01 00:00:00.091666    1

## Pandas DataFrame

Converting to a pandas dataframe will produce a tabular view of the data similar to calling the `rows` method.  The resulting dataframe will be indexed by time.

In [15]:
streamset.filter(start, end).to_dataframe()

,sunshine/PMU2/C2MAG,sunshine/PMU2/C1MAG,sunshine/PMU2/C3MAG
time,,,
1456790400008333000,115.008881,115.584435,104.032997
1456790400016666000,114.932472,115.833641,103.941956
1456790400024999000,115.066551,116.065895,103.556023
1456790400033333000,115.177299,115.894745,103.057266
1456790400041666000,114.992668,115.308258,102.640678
1456790400049999000,114.456436,114.271133,102.258713
1456790400058333000,113.854378,113.307884,102.027412
1456790400066666000,113.729301,113.053108,102.049446
1456790400074999000,113.935654,113.111473,102.063484


## Python Dictionaries

Converting to Python dictionaries produces a list of `OrderedDicts` similar to calling the `rows` method. 

In [16]:
streamset.filter(start, end).to_dict()

[OrderedDict([('time', 1456790400008333000),
              ('sunshine/PMU2/C2MAG', 115.00888061523438),
              ('sunshine/PMU2/C1MAG', 115.58443450927734),
              ('sunshine/PMU2/C3MAG', 104.03299713134766)]),
 OrderedDict([('time', 1456790400016666000),
              ('sunshine/PMU2/C2MAG', 114.9324722290039),
              ('sunshine/PMU2/C1MAG', 115.8336410522461),
              ('sunshine/PMU2/C3MAG', 103.94195556640625)]),
 OrderedDict([('time', 1456790400024999000),
              ('sunshine/PMU2/C2MAG', 115.0665512084961),
              ('sunshine/PMU2/C1MAG', 116.0658950805664),
              ('sunshine/PMU2/C3MAG', 103.55602264404297)]),
 OrderedDict([('time', 1456790400033333000),
              ('sunshine/PMU2/C2MAG', 115.17729949951172),
              ('sunshine/PMU2/C1MAG', 115.89474487304688),
              ('sunshine/PMU2/C3MAG', 103.05726623535156)]),
 OrderedDict([('time', 1456790400041666000),
              ('sunshine/PMU2/C2MAG', 114.99266815185547),
    

# Serializing Data

Aside from transforming to other data objects, you can also serialize the data to other formats.  At the moment, you can serialize the data a string for display.


In [17]:
# create a new streamset with only 2 streams for better display in a notebook
streamset = conn.streams(*UUIDs[:2])

# call to_csv which will detect the output stream and write the data to it
print(streamset.filter(start, end).to_table())

               time    sunshine/PMU2/C2MAG    sunshine/PMU2/C1MAG
-------------------  ---------------------  ---------------------
1456790400008333000                115.009                115.584
1456790400016666000                114.932                115.834
1456790400024999000                115.067                116.066
1456790400033333000                115.177                115.895
1456790400041666000                114.993                115.308
1456790400049999000                114.456                114.271
1456790400058333000                113.854                113.308
1456790400066666000                113.729                113.053
1456790400074999000                113.936                113.111
1456790400083333000                113.954                113.043
1456790400091666000                113.692                112.812
1456790400099999000                113.298                112.506
